In [1]:
from module import connection
from module import get
from module import update
from module import search
from module import save

In [2]:
save.save_min_day_concat()

0it [00:00, ?it/s]


KeyError: 20221103